In [ ]:
import os
import sys
import re
import time
import random
import matplotlib
import matplotlib.pyplot as plt

parentPath = os.path.abspath("../..")
if parentPath not in sys.path:
    sys.path.insert(0, parentPath)

import model as modellib
import visualize
from model import log

%matplotlib inline 

# Root directory of the project
ROOT_DIR = parentPath

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Path to COCO trained weights
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")

ADE20K_DIR = "/home/orestisz/data/ADE20K_2016_07_26"
COCO_DIR = "/home/orestisz/repositories/coco"
SCENENN_DIR = "/home/orestisz/data/sceneNN"

In [ ]:
import coco

# import ADE20K
from dataset import *

In [ ]:
def get_ax(rows=1, cols=1, size=8):
    """Return a Matplotlib Axes array to be used in
    all visualizations in the notebook. Provide a
    central point to control graph sizes.
    
    Change the default size attribute to control the size
    of rendered images
    """
    _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    return ax

In [ ]:
# # coco dataset
dataset_coco = coco.CocoDataset()
dataset_coco.load_coco(COCO_DIR, "minival")
dataset_coco.prepare()

# Validation dataset
dataset_val = Dataset()
dataset_val.load(ADE20K_DIR, "validation")
dataset_val.prepare()

# import object_segmentation.sceneNN.dataset as dataset_sceneNN
# dataset_sceneNN_test = dataset_sceneNN.Dataset()
# dataset_sceneNN_test.load(SCENENN_DIR, "testing")
# dataset_sceneNN_test.prepare()

# dataset = dataset_sceneNN_test
dataset = dataset_val

## Detection

In [ ]:
# class InferenceConfig(dataset_sceneNN.Config):
class InferenceConfig(Config):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
class InferenceConfigCoco(coco.CocoConfig):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

inference_config = InferenceConfig()
inference_config_coco = InferenceConfigCoco()
# Recreate the model in inference mode
model = modellib.MaskRCNN(mode="inference", config=inference_config, model_dir=MODEL_DIR)
model_coco = modellib.MaskRCNN(mode="inference", config=inference_config_coco, model_dir=MODEL_DIR)

In [ ]:
# Test on a random image
image_id = random.choice(dataset.image_ids)
original_image, image_meta, class_ids, bbox, mask =\
    modellib.load_image_gt(dataset, inference_config, 
                           image_id, use_mini_mask=False)
    
log("original_image", original_image)
log("image_meta", image_meta)
log("bbox", bbox)
log("mask", mask)
visualize.display_top_masks(original_image, mask, class_ids, dataset.class_names, limit=1)

visualize.display_instances(original_image, bbox, mask, class_ids, 
                            dataset.class_names, figsize=(8, 8))

In [ ]:
# coco model
print("Loading weights from Coco Model with all classes")
model_coco.load_weights(COCO_MODEL_PATH, by_name=True)
r = model_coco.detect([original_image])[0]
visualize.display_instances(original_image, r['rois'], r['masks'], r['class_ids'], 
                            dataset_coco.class_names, r['scores'], title='coco', ax=get_ax())

skip = 3
model_paths = model.find_all()
for model_path in model_paths:
    print("Loading weights from ", model_path[0])
    for i, checkpoint in enumerate(model_path[1]):
        if i % (skip + 1) == 0:
            print("    " + checkpoint)
            model.load_weights(checkpoint, by_name=True)
            start = time.clock()
            r = model.detect([original_image])[0]
            print(time.clock() - start)
            visualize.display_instances(original_image, r['rois'], r['masks'], r['class_ids'], 
                                dataset.class_names, r['scores'], title=checkpoint, ax=get_ax())

## Evaluation

In [ ]:
# Compute VOC-Style mAP @ IoU=0.5
# Running on 10 images. Increase for better accuracy.
image_ids = np.random.choice(dataset.image_ids, 20)
APs = []
for i, image_id in enumerate(image_ids):
    # Load image and ground truth data
    try:
        image_path = dataset.image_info[image_id]['path']
        image, image_meta, gt_class_id, gt_bbox, gt_mask =\
            modellib.load_image_gt(dataset, inference_config,
                                   image_id, use_mini_mask=False)
        # Run object detection
        results = model.detect([image], verbose=0)
        r = results[0]
        # Compute AP
        AP, precisions, recalls, overlaps =\
            utils.compute_ap(gt_bbox, gt_class_id, r["rois"], r["class_ids"], r["scores"])
        APs.append(AP)
        print('{}: {}: {}'.format(i + 1, image_path, AP))
    except:
        print("Error processing image {}".format(image_path))
    
print("mAP: ", np.mean(APs))